In [62]:
from striprtf.striprtf import rtf_to_text
import json

with open(r"C:\Users\91962\Downloads\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui.json.rtf", "r", encoding="utf-8") as file:
    rtf_content = file.read()
    
text = rtf_to_text(rtf_content)
dic = json.loads(text)

In [63]:
dic

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(whole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0.7,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_nam

In [64]:
dic.keys()

dict_keys(['session_name', 'session_description', 'design_state_data'])

In [65]:
from striprtf.striprtf import rtf_to_text
import json

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [66]:
class preprocess_pipeline():
    def __init__(self, addrs):
        with open(addrs, "r", encoding="utf-8") as file:
            rtf_content = file.read()
        text = rtf_to_text(rtf_content)
        self.file = json.loads(text)
        self.hashed = []
        
    def read_df(self):
        file = self.file["design_state_data"]["session_info"]["dataset"]
        df = pd.read_csv(file)
        print("READ THE DATAFRAME")
        return df
        
    def data_selection(self):
        self.df = self.read_df()
        # target vars
        pred = self.file["design_state_data"]["target"]["prediction_type"]
        self.target = self.file["design_state_data"]["target"]["target"]
        type_ = self.file["design_state_data"]["target"]["type"]
        partition = self.file["design_state_data"]["target"]["partitioning"]

        # train vars
        train_columns = [x for x in self.df.columns if x != self.target]
        train_size = self.file["design_state_data"]["train"]["train_ratio"]
        
        policy = self.file["design_state_data"]["train"]["policy"]
        split1 = "random" in self.file["design_state_data"]["train"]["policy"].lower()
        split2 = "strat" in self.file["design_state_data"]["train"]["policy"].lower()
        randomness = self.file["design_state_data"]["train"]["random_seed"]
        KFOLD = self.file["design_state_data"]["train"]["k_fold"]
        
        # train test split
        if "split" in policy.lower():
            X_train, X_test, Y_train, Y_test = train_test_split(
                                                                self.df[train_columns], 
                                                                self.df[self.target], 
                                                                train_size = train_size, 
                                                                shuffle = split1,
                                                                #stratify = split2,
                                                                random_state = randomness 
                                                                )
            
            X_train, X_test, Y_train, Y_test = self.feature_handling(X_train, X_test, Y_train, Y_test)
            X_train, X_test = self.feature_generation(X_train, X_test)
            print("SPLIT IMPUTATION COMPLETE")
            return "normal_split", [X_train, X_test, Y_train, Y_test]
            
        if KFOLD:
            kf = KFold(n_splits=2, shuffle=True, random_state=42)
            folds = []
            # Generate splits
            for train_idx, test_idx in kf.split(df):
                X_train = df.loc[train_idx, train_columns]
                X_test = df.loc[test_idx, train_columns]
                Y_train = df.loc[train_idx, target]
                Y_test = df.loc[test_idx, target]
                
                X_train, X_test, Y_train, Y_test = self.feature_handling(X_train, X_test, Y_train, Y_test)
                X_train, X_test = self.feature_generation(X_train, X_test)
                folds.append([X_train, X_test, Y_train, Y_test])
                
            print("KFOLDED IMPUTATION COMPLETE")
            return "kfold", folds
    
    def feature_handling(self, X_train, X_test, Y_train, Y_test):
        # list features
        features = list(self.file["design_state_data"]["feature_handling"].keys())
        # grab the feature dictionary part
        features_dic = self.file["design_state_data"]["feature_handling"]
        #print(features_dic)
        for feature in features:
            if features_dic[feature]["is_selected"]:
                if features_dic[feature]["feature_variable_type"].lower() == "numerical":
                    imp_method = features_dic[feature]["feature_details"]["impute_with"].lower()
                    if imp_method == "custom":
                        imp_value = features_dic[feature]["feature_details"]["impute_value"]
                        if feature == self.target:
                            Y_train.fillna(imp_value,  inplace = True)
                            Y_test.fillna(imp_value,  inplace = True) 
                        else:
                            X_train.fillna({feature : imp_value}, inplace = True)
                            X_test.fillna({feature : imp_value},  inplace = True)
    
                    else:
                        if features_dic[feature]["feature_variable_type"].lower() == "numerical":
                            if "mean" in imp_method or "average" in imp_method:
                                strategy = "mean"
                            elif "median" in imp_method:
                                strategy = "median"
                            elif "mode" in imp_method:
                                strategy = "mode"
                            si = SimpleImputer(strategy = strategy)
    
                            if feature == self.target:
                                si.fit(Y_train)
                                Y_train = si.transform(Y_train)
                                Y_test = si.transform(Y_test)
                            else:
                                si.fit(X_train[feature].values.reshape(-1, 1))
                                X_train[feature] = si.transform(X_train[feature].values.reshape(-1, 1)).ravel()
                                X_test[feature] = si.transform(X_test[feature].values.reshape(-1, 1)).ravel()
                                
                else: # for text data
                    self.hashed.append(feature)
                    strategy = "most_frequent"
    
                    si = SimpleImputer(strategy = strategy)
    
                    if feature == self.target:
                        si.fit(Y_train.values.reshape(-1, 1))
                        Y_train = si.transform(Y_train.values.reshape(-1, 1)).ravel()
                        Y_test = si.transform(Y_test.values.reshape(-1, 1)).ravel()
                    else:
                        si.fit(X_train[feature].values.reshape(-1, 1))
                        X_train[feature] = si.transform(X_train[feature].values.reshape(-1, 1)).ravel()
                        X_test[feature] = si.transform(X_test[feature].values.reshape(-1, 1)).ravel()
                    """
                    not expecting the target variable to be hashed or tokenized so ignoring y_train, 
                    but if it is, we will use Label encoder instead
                    """
                    if feature == self.target:
                        e = LabelEncoder()
                        e.fit(Y_train)
                        Y_train = e.transform(Y_train)
                        Y_test = e.transform(Y_test)
                        
                    else:
                        # use hashing evctorizer
                        hash_columns = features_dic[feature]["feature_details"]["hash_columns"]
                        if hash_columns != 0: 
                            vectorizer = HashingVectorizer(n_features = hash_columns, stop_words = 'english', alternate_sign = False)
            
                            hashed_X_train = vectorizer.transform(X_train[feature].astype(str)).toarray()
                            hashed_X_test = vectorizer.transform(X_test[feature].astype(str)).toarray()
                            
                            # vectorizer returns dense array create columns out of that 
                            hashed_X_train_df = pd.DataFrame(hashed_X_train, columns=[f"hashed_{feature}_{i}" for i in range(hashed_X_train.shape[1])])
                            hashed_X_test_df = pd.DataFrame(hashed_X_test, columns=[f"hashed_{feature}_{i}" for i in range(hashed_X_test.shape[1])])
                            # join
                            X_train = pd.concat([X_train, hashed_X_train_df], axis=1)
                            X_test = pd.concat([X_test, hashed_X_test_df], axis=1)

                            # drop the string col
                            X_train.drop(feature, axis = 1, inplace = True)
                            X_test.drop(feature, axis = 1, inplace = True)
                            
                        else:
                            e = LabelEncoder()
                            e.fit(X_train[feature])
                            X_train[feature] = e.transform(X_train[feature].values.reshape(-1, 1)).flatten()
                            X_test[feature]= e.transform(X_test[feature].values.reshape(-1, 1)).flatten()
        
        return X_train, X_test, Y_train, Y_test  

    def interaction(self, interaction_list, interaction_type, X_train, X_test):
        """
        handles interaction and takes care of hashed array interactions if any 
        """
        functiondic = {
                       "linear" : lambda x, y : x + y,
                       "poly" : lambda x, y : x / y, 
                       "expl" : lambda x, y : x / y
                      }
        
        function = functiondic[interaction_type]
        for x, y in interaction_list:
            # skip when target is given for interaction, this causes dataleakage
            if y == self.target: continue
            if x == self.target: continue
            
            if x in self.hashed:
                cols = [p for p in X_train.columns if f"hashed_{x}" in p]
                for hashed_col in cols:
                    X_train[f"{hashed_col}_l_{y}"] = function(X_train[hashed_col], X_train[y])
                    X_test[f"{hashed_col}_l_{y}"] = function(X_test[hashed_col], X_test[y])

            elif y in self.hashed:
                cols = [p for p in X_train.columns if f"hashed_{y}" in p]
                for hashed_col in cols:
                    X_train[f"{x}_l_{hashed_col}"] = function(X_train[x], X_train[hashed_col])
                    X_test[f"{x}_l_{hashed_col}"] = function(X_test[x], X_test[hashed_col])
            
            # not expecting both to be hashed would be worst case scenario
            elif x in self.hashed and y in self.hashed:
                colx = [p for p in X_train.columns if f"hashed_{x}" in p]
                coly = [p for p in X_train.columns if f"hashed_{y}" in p]
                
                for hashed_x in colx:
                    for hashed_y in coly:
                        X_train[f"{hashed_x}_l_{hashed_y}"] = function(X_train[hashed_x], X_train[hashed_y])
                        X_test[f"{hashed_x}_l_{hashed_y}"] = function(X_test[hashed_x], X_test[hashed_y])
            else:
                X_train[f"{x}_l_{y}"] = function(X_train[x], X_train[y])
                X_test[f"{x}_l_{y}"] = function(X_test[x], X_test[y])
                
        return X_train, X_test
        

    def feature_generation(self, X_train, X_test):
        dic = self.file["design_state_data"]["feature_generation"]
        
        # add new linear features
        X_train, X_test = self.interaction(dic["linear_interactions"], "linear", X_train, X_test)
            
        # add polynomial features
        polynomial_interactions = [x.split("/") for x in dic["polynomial_interactions"]]
        X_train, X_test = self.interaction(polynomial_interactions, "poly", X_train, X_test)
        
        # add explicit pariwaise reln
        explicit_pairwise_interactions = [x.split("/") for x in dic["explicit_pairwise_interactions"]]
        X_train, X_test = self.interaction(explicit_pairwise_interactions, "expl", X_train, X_test)

        return X_train, X_test

In [85]:
k = preprocess_pipeline(r"C:\Users\91962\Downloads\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui.json.rtf")
u = k.data_selection()

READ THE DATAFRAME
SPLIT IMPUTATION COMPLETE


In [86]:
class feature_reduction_pipeline:
    def __init__(self, dic, X_train, X_test, target):
        self.dic = dic
        self.method = dic["feature_reduction_method"].lower()
        self.itrain = X_train.index
        self.itest = X_test.index
        self.df = pd.concat([X_train, X_test], axis = 0)
        self.target = target
    
    def execute(self):
        if "correlation" in self.method:
            return self.correlation(0.4)
            
        elif "principal" in self.method:
            return self.pca(self.dic[self.method]["num_features_to_keep"])
            
        elif "tree" in self.method:
            depth = self.dic[self.method]["depth_of_trees"]
            numt = self.dic[self.method]["num_of_trees"]
            n = self.dic[self.method]["num_features_to_keep"]
            return self.tree(self.df, num_features_to_keep = n, num_trees = numt, depth = depth)
        
        else:
            return self.df.iloc[self.itrain], self.df.iloc[self.itest]
            
    def correlation(self, threshold = 0.4):
        corr_matrix = self.df.corr().abs()
        # upper triangle of correlation matrix
         # Keeps the values where the condition is True and replaces everything else with NaN.                  
        """                 |    # makes it triangle # creates a corelation matrix like array with ones
                            |           |                |
                            v           v                v                 """
        """
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if upper.iloc[i, j] >= threshold:
                    to_drop.append(corr_matrix.columns[i])
        keep = [column for column in corr_matrix.columns if column not in to_drop]
        return self.df.loc[self.itrain, keep], self.df.loc[self.itest, keep]
        """

        corr_matrix = self.df.corr().abs()
        # Upper triangle of correlation matrix
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = []
        for column in upper.columns:
            # Find columns where correlation exceeds threshold
            if any(upper[column] >= threshold):
                to_drop.append(column)
                
        keep = [column for column in corr_matrix.columns if column not in to_drop]
        return self.df.loc[self.itrain, keep], self.df.loc[self.itest, keep]

    def pca(self, n = 3):
        pca = PCA(n_components=num_features_to_keep)
        df_pca = pca.fit_transform(self.df)
        return df_pca.iloc[self.itrain], df_pca.iloc[self.itest]

    def tree(df, num_features_to_keep = 10, num_trees = 100, depth = 6):
        # fit an rf model
        rf = RandomForestClassifier(n_estimators = num_trees, max_depth = depth)
        y = self.df[self.target]
        X = self.df[[x for x in self.df.columns if x != self.target]]
        rf.fit(X, y)
        # select top features using feature importances
        importances = rf.feature_importances_
        feature_names = X.columns
    
        # sort by importance and select top N
        sorted_indices = importances.argsort()[::-1][:num_features_to_keep]
        selected_features = feature_names[sorted_indices]
    
        # subset the data
        X_train_reduced = self.df.loc[self.itrain, selected_features]
        X_test_reduced = self.df.loc[self.itest, selected_features]
    
        return X_train_reduced, X_test_reduced


In [92]:
class feature_reduction_pipeline:
    def __init__(self, dic, X_train, X_test, target):
        self.dic = dic
        self.method = dic["feature_reduction_method"].lower()
        self.X_train = X_train.copy()
        self.X_test = X_test.copy()
        self.target = target
    
    def execute(self):
        if "correlation" in self.method:
            return self.correlation(0.4)
            
        elif "principal" in self.method:
            return self.pca(self.dic[self.method]["num_features_to_keep"])
            
        elif "tree" in self.method:
            depth = self.dic[self.method]["depth_of_trees"]
            numt = self.dic[self.method]["num_of_trees"]
            n = self.dic[self.method]["num_features_to_keep"]
            return self.tree(num_features_to_keep=n, num_trees=numt, depth=depth)
        
        else:
            return self.X_train, self.X_test
            
    def correlation(self, threshold=0.4):
        combined = pd.concat([self.X_train, self.X_test])
        corr_matrix = combined.corr().abs()
        
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = []
        
        for column in upper.columns:
            if any(upper[column] >= threshold):
                to_drop.append(column)
                
        keep = [column for column in corr_matrix.columns if column not in to_drop]
        
        return self.X_train[keep], self.X_test[keep]
    
    def pca(self, n=3):
        combined = pd.concat([self.X_train, self.X_test])
        
        pca = PCA(n_components=n)
        pca.fit(combined)
        
        train_transformed = pca.transform(self.X_train)
        test_transformed = pca.transform(self.X_test)
        
        cols = [f'PC{i+1}' for i in range(n)]
        train_pca = pd.DataFrame(train_transformed, index = self.X_train.index, columns = cols)
        test_pca = pd.DataFrame(test_transformed, index = self.X_test.index, columns = cols)
        
        return train_pca, test_pca
    
    def tree(self, num_features_to_keep=10, num_trees=100, depth=6):
        if self.target in self.X_train.columns:
            X = self.X_train.drop(columns=[self.target])
            y = self.X_train[self.target]
        else:
            X = self.X_train
            raise ValueError("Target column not found in X_train. Please ensure target is available.")
        
        rf = RandomForestClassifier(n_estimators = num_trees, max_depth = depth)
        rf.fit(X, y)
        
        # Get feature importances
        importances = rf.feature_importances_
        feature_names = X.columns.tolist()
        
        # Sort features by importance
        features_with_importance = list(zip(feature_names, importances))
        sorted_features = sorted(features_with_importance, key=lambda x: x[1], reverse=True)
        
        # Select top N features
        selected_features = [f[0] for f in sorted_features[:num_features_to_keep]]
        
        # Subset the data - ensuring we only use available columns
        X_train_reduced = self.X_train[selected_features]
        X_test_reduced = self.X_test[selected_features]
        
        return X_train_reduced, X_test_reduced

In [93]:
local_dic = dic["design_state_data"]["feature_reduction"]
target = dic["design_state_data"]["target"]["target"]
dimred = feature_reduction_pipeline(local_dic, u[1][0], u[1][1], target)
k = dimred.execute()

In [94]:
k[0]

,sepal_length,sepal_width,hashed_species_0,hashed_species_2,hashed_species_3,hashed_species_4,hashed_species_5,hashed_species_6,hashed_species_8,hashed_species_9
0,5.1,3.5,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
1,4.9,3.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
2,4.7,3.2,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
3,4.6,3.1,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
4,5.0,3.6,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
100,6.3,3.3,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
101,5.8,2.7,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
102,7.1,3.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
103,6.3,2.9,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
